# PRÁCTICA GUIADA : Curva ROC y Area bajo la curva

## 1. Introducción

Seguimos trabajando sobre el dataset de RRHH. La variable dependiente es la misma ($P(left=1|X)$)

In [ ]:
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_curve, auc, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('../Data/HR_comma_sep.csv')
df.sample(10)

In [ ]:
train_cols = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 
              'time_spend_company', 'Work_accident', 'promotion_last_5years']
X = df[train_cols]
y = df['left']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

* Definimos y entrenamos el modelo (Regresión Logística)

In [ ]:
clf = LogisticRegression(C=1e10)
clf.fit(X_train, y_train)

## 2. Ajustando los umbrales

Hasta ahora siempre hemos trabajado asumiendo que si $p(y=1) > 0.5$, entonces, la predicción del modelo será que $y=1$. Ahora bien, ¿qué sucede si queremos maximizar o modificar la performance del modelo en alguna de las métricas que hemos visto antes (sentivity, recall, etc.)? ¿Cómo podemos lograr esto?

Una forma es haciendo variar esa regla que habíamos definido más arriba: ajustar los umbrales.

Veamos cómo funciona. En primer lugar, obtengamos las predicciones de probabilidad ($p(y=1)$) y no las predicciones de la clase de $y$.

In [ ]:
clf.predict(X_test)

In [ ]:
clf.predict_proba(X_test)

Notar que el método `predict_proba` nos devuelve una array en el cual aparecen dos probabilidades de cada instancia del test set: $p(y=0)$ y $p(y=1)$, en ese orden.

`sklearn` realiza la predicción de la clase de $y$ eligiendo para cada clase la mayor probabilidad de este array.

In [ ]:
print(np.mean(clf.predict_proba(X_test)[:,1] > 0.5))
print(np.mean(clf.predict(X_test)))

y_pred_orig = clf.predict(X_test)

Guardemos las probabilidades de ambas clases en un array y $p(y=1)$ en otro:

In [ ]:
y_probs_logit = clf.predict_proba(X_test)
y_probs_logit_left = y_probs_logit[:,1]

In [ ]:
# 15 bins
plt.hist(y_probs_logit_left, bins=15)

# x-axis de 0 a 1
plt.xlim(0,1)
plt.title('Histograma de probabilidades estimadas')
plt.xlabel('Probabilidad estimada de dejar la empresa')
plt.ylabel('Frecuencia')
plt.show()

* La gran mayoría de las probabilidades predichas van de 0.0 a 0.4
* Hay un escaso número de probabilidades estimadas mayores a 0.5
* La mayor parte de los casos van a ser predichos como $y=0$ es decir, que no se van de la empresa.

Una posible solución es, entonces, variar el umbral -$y = 1$ si $p(y=1) > 0.5$-. 

En este caso, lo lógico sería bajar el umbral. Por lo tanto se incrementará la sensitivity. ¿Por qué?

* Aumentaremos la cantidad de TP
* El clasificador será más "sensible" a las instancias posiitvas

In [ ]:
from sklearn.preprocessing import binarize
y_pred_logit = binarize(y_probs_logit, 0.3)[:,1]

# Otra forma
#y_pred_logit = (y_probs_logit_left > 0.3)

* Veamos la matriz de confusión con las predicciones basadas en el modelo original

In [ ]:
confusion_matrix(y_test,y_pred_orig)

* Veamos la matriz de confusión con las predicciones basadas en el modelo modificado

In [ ]:
confusion_matrix(y_test,y_pred_logit)

* ¿Qué pasó con la sensitividad (o recall)? 

$\large recall = \frac{TP}{(FN + TP)}$

In [ ]:
print('Recall umbral 0.5=', recall_score(y_test, y_pred_orig))
print('Recall umbral 0.3=', recall_score(y_test, y_pred_logit))

* ¿Qué pasó con la specificity? 

$\large specificity = \frac{TN}{(TN + FP)}$

In [ ]:
def specificy(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    specificity = tn / (tn+fp)
    return(specificity)

In [ ]:
print('Spec umbral 0.5 =', specificy(y_test, y_pred_orig))
print('Spec umbral 0.3 =', specificy(y_test, y_pred_logit))

In [ ]:
print('Acc umbral 0.5 =', accuracy_score(y_test, y_pred_orig))
print('Acc umbral 0.3 =', accuracy_score(y_test, y_pred_logit))

In [ ]:
from sklearn.metrics import f1_score
print('F1 score 0.5 =', f1_score(y_test, y_pred_orig))
print('F1 score 0.3 =', f1_score(y_test, y_pred_logit))

* Se puede ajustar el umbral para las predicciones en clasificadores binarios
* El ajuste de este umbral repercute sobre las diferentes medidas de performance.
* Particularmente, sensitivity y specificity tiene una relación inversa
    * siempre al mejorar uno, empeorará el otro

## 3. Curvas ROC y área bajo la curva (AUC)

Muy útil si queremos visualizar cómo se mueven sensitivity y specificity ante diversos umbrales. 

La curva ROC se basa en $TPR$ (tasa de verdaderos positivos) y $FPR$ (tasa de falsos negativos).

* Definamos las metricas de True Positive Ratio y False Positive Rate y se las asignamos las los valores predichos vs los valores de test (observados vs esperados).
* El método `roc_curve` toma como parámetros dos valores: los valores observados del target y un array de probabilidades (NO recibe las predicciones de la clase).
* Devuelve tres elementos en forma de arrays: la tasa de falsos positivos ($FPR$), la tasa de verdaderos positivos ($TPR$) y los umbrales

In [ ]:
fpr_log,tpr_log,thr_log = roc_curve(y_test, y_probs_logit[:,1])

* Convertimos los valores en un objeto dataframe y graficamos la curva ROC

In [ ]:
df = pd.DataFrame(dict(fpr=fpr_log, tpr=tpr_log, thr = thr_log))

In [ ]:
plt.axis([0, 1.01, 0, 1.01])
plt.xlabel('FPR = 1 - Specificty')
plt.ylabel('TPR = Sensitivity = Recall')
plt.title('ROC Curve')
plt.plot(df['fpr'],df['tpr'])
plt.plot(np.arange(0,1, step =0.01), np.arange(0,1, step =0.01))
plt.show() 

* Calculamos el área bajo la curva ROC

In [ ]:
print('AUC=', auc(fpr_log, tpr_log))

## 4. PRÁCTICA INDEPENDIENTE

Evaluar las medidas vistas en ambas prácticas para un clasificador Naïve Bayes.

In [ ]:
from sklearn.naive_bayes import GaussianNB

model_nb = GaussianNB()
model_nb.fit(X_train, y_train)
y_fit_nb = model_nb.predict_proba(X_test)

In [ ]:
fpr_log,tpr_log,thr_log = roc_curve(y_test, y_fit_nb[:,1])

In [ ]:
df = pd.DataFrame(dict(fpr=fpr_log, tpr=tpr_log, thr = thr_log))

plt.axis([0, 1.01, 0, 1.01])
plt.xlabel('FPR = 1 - Specificty')
plt.ylabel('TPR = Sensitivity = Recall')
plt.title('ROC Curve')
plt.plot(df['fpr'],df['tpr'])
plt.plot(np.arange(0,1, step =0.01), np.arange(0,1, step =0.01))
plt.show() 

In [ ]:
print('AUC=', auc(fpr_log, tpr_log))

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

scores = []

for i in range(1,20,1):
    model = KNeighborsClassifier(n_neighbors=i) 
    cv_scores = cross_val_score(model, X_test, y_test, cv=5)

    scores.append({
        'score_mean': np.mean(cv_scores),
        'score_std': np.std(cv_scores),
        'n': i
    })

scores_knn = pd.DataFrame(scores)
scores_knn['low'] = scores_knn['score_mean'] - scores_knn['score_std']
scores_knn['high'] = scores_knn['score_mean'] + scores_knn['score_std']

import matplotlib.pyplot as plt
%matplotlib inline

plt.plot(scores_knn['n'], scores_knn['low'], color='r')
plt.plot(scores_knn['n'], scores_knn['score_mean'], color='b')
plt.plot(scores_knn['n'], scores_knn['high'], color='r');

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=2)
knn_model.fit(X_train, y_train)
y_fit_knn = knn_model.predict_proba(X_test)
fpr_log,tpr_log,thr_log = roc_curve(y_test, y_fit_knn[:,1])

df = pd.DataFrame(dict(fpr=fpr_log, tpr=tpr_log, thr = thr_log))

plt.axis([0, 1.01, 0, 1.01])
plt.xlabel('FPR = 1 - Specificty')
plt.ylabel('TPR = Sensitivity = Recall')
plt.title('ROC Curve')
plt.plot(df['fpr'],df['tpr'])
plt.plot(np.arange(0,1, step =0.01), np.arange(0,1, step =0.01))
plt.show() 

In [ ]:
print('AUC=', auc(fpr_log, tpr_log))

In [ ]:
y_fit_knn = knn_model.predict(X_test)
confusion_matrix(y_test,y_fit_knn)

## SVM

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

from sklearn.svm import SVC
model_svc = SVC(probability=True)
model_svc.fit(X_train, y_train)
y_fit_svc = model_svc.predict_proba(X_test)

fpr_log,tpr_log,thr_log = roc_curve(y_test, y_fit_svc[:,1])

df = pd.DataFrame(dict(fpr=fpr_log, tpr=tpr_log, thr = thr_log))

plt.axis([0, 1.01, 0, 1.01])
plt.xlabel('FPR = 1 - Specificty')
plt.ylabel('TPR = Sensitivity = Recall')
plt.title('ROC Curve')
plt.plot(df['fpr'],df['tpr'])
plt.plot(np.arange(0,1, step =0.01), np.arange(0,1, step =0.01))
plt.show() 

In [ ]:
print('AUC=', auc(fpr_log, tpr_log))